# Hybrid pipeline (Option 2) — Research-grade skeleton

KAN → VQE → Krylov/SBQD → FFT → QPE

This notebook is a modular research-grade skeleton for Colab/QBraid. It includes hardware-ready builders and placeholders for runtime submission. It is intended for extension and experimentation.

## Setup: install & imports

In [ ]:
!pip install --quiet qiskit qiskit-aer qiskit-ibm-runtime torch scipy numpy matplotlib nbformat

In [ ]:
import os, json, math, time
import numpy as np
import torch
from torch import nn
from scipy.linalg import expm, eig, fractional_matrix_power
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.primitives import Estimator
from qiskit.quantum_info import Operator, SparsePauliOp
from qiskit.extensions import UnitaryGate
from qiskit.algorithms.optimizers import COBYLA

try:
    from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as RuntimeEstimator, SamplerV2 as RuntimeSampler
    print('Runtime available')
except Exception:
    QiskitRuntimeService = None
    RuntimeEstimator = None
    RuntimeSampler = None
    print('Runtime not available; defaulting to local simulation')

## Build Hamiltonian & Pauli decomposition

In [ ]:
# Build H_small (adjust N_TRUNC for research)
N_TRUNC = 10
N_QUBITS = int(np.ceil(np.log2(N_TRUNC)))
DIM = 2**N_QUBITS

def ladder_ops(N):
    a = np.zeros((N,N), dtype=complex)
    for n in range(N-1):
        a[n,n+1] = np.sqrt(n+1)
    return a, a.conj().T

a, adag = ladder_ops(N_TRUNC)
x = (a+adag)/np.sqrt(2)
p = 1j*(adag - a)/np.sqrt(2)
H_small = 0.5*(x@p + p@x)
H_small = 0.5*(H_small + H_small.conj().T)
H_pad = np.zeros((DIM,DIM), dtype=complex)
H_pad[:N_TRUNC, :N_TRUNC] = H_small
H_op = Operator(H_pad)
print('Built H with N_TRUNC=', N_TRUNC)

## Pauli decomposition and Trotter builder (caution: scales as 4^n)

In [ ]:
from qiskit.quantum_info import SparsePauliOp
paulis = ['I','X','Y','Z']
pauli_terms = []
for idx in range(4**N_QUBITS):
    label = ''
    tmp = idx
    for _ in range(N_QUBITS):
        label = paulis[tmp % 4] + label
        tmp //= 4
    sp = SparsePauliOp.from_list([(label,1.0)])
    mat = sp.to_matrix()
    c = np.trace(mat.conj().T @ H_pad) / (2**N_QUBITS)
    if abs(c) > 1e-12:
        pauli_terms.append((label, float(c)))
print('Nonzero Pauli terms:', len(pauli_terms))

def pauli_term_to_exp_circ(label, angle):
    qc = QuantumCircuit(N_QUBITS)
    for q, p in enumerate(reversed(label)):
        if p == 'X': qc.h(q)
        elif p == 'Y': qc.sdg(q); qc.h(q)
    for q in range(1, N_QUBITS): qc.cx(q, 0)
    qc.rz(2*angle, 0)
    for q in range(1, N_QUBITS): qc.cx(q, 0)
    for q, p in enumerate(reversed(label)):
        if p == 'X': qc.h(q)
        elif p == 'Y': qc.h(q); qc.s(q)
    return qc

def trotter_controlled(pauli_terms, t, r):
    qc = QuantumCircuit(1 + N_QUBITS)
    dt = t / r
    for _ in range(r):
        for label, coeff in pauli_terms:
            angle = coeff * dt
            base = pauli_term_to_exp_circ(label, angle)
            qc.append(base.to_gate().control(1), [0] + list(range(1,1+N_QUBITS)))
    return qc

print('Trotter builder ready')

## KAN + VQE (research)

Reuse the KAN surrogate and Estimator patterns from Option 1, but increase budgets and logging for research experiments.

In [ ]:
class KANSurrogate(nn.Module):
    def __init__(self, in_dim, m_units=4*in_dim, uni_hidden=16):
        super().__init__()
        self.inner = nn.Linear(in_dim, m_units)
        self.uni_w = nn.Parameter(torch.randn(m_units, uni_hidden) * 0.01)
        self.uni_b = nn.Parameter(torch.zeros(m_units, uni_hidden))
        self.uni_out = nn.Parameter(torch.randn(m_units) * 0.01)
        self.outer = nn.Linear(m_units, 1)
    def forward(self, x):
        z = self.inner(x)
        z_exp = z.unsqueeze(-1)
        hidden = torch.tanh(z_exp * self.uni_w.unsqueeze(0) + self.uni_b.unsqueeze(0))
        hid_mean = hidden.mean(dim=-1)
        uni = hid_mean * self.uni_out.unsqueeze(0)
        out = self.outer(uni)
        return out.squeeze(-1)

sim = AerSimulator(method='statevector')
est = Estimator(sim)

n_params = 2 * N_QUBITS
print('KAN+VQE building blocks ready')

## SBQD / Krylov measurement on hardware — SWAP and Hadamard tests (skeleton)

In [ ]:
def hadamard_test(prep_circ, unitary_circ, measure_imag=False):
    qc = QuantumCircuit(1 + N_QUBITS, 1)
    qc.h(0)
    if measure_imag: qc.sdg(0)
    qc.compose(prep_circ, qubits=list(range(1,1+N_QUBITS)), inplace=True)
    qc.append(unitary_circ.to_gate().control(1), [0] + list(range(1,1+N_QUBITS)))
    qc.h(0)
    qc.measure(0,0)
    return qc

print('Hadamard-test skeleton ready')

## FFT step & QPE skeleton (research)

In [ ]:
def compute_time_series_statevector(psi0, U_step, n_time=256):
    C = np.zeros(n_time, dtype=complex)
    cur = psi0.copy()
    for n in range(n_time):
        C[n] = np.vdot(psi0, cur)
        cur = U_step @ cur
    return C

print('FFT & QPE helpers ready')

## Next steps and how to run on hardware

1. Fill in the active KAN training loop similar to Option 1 but with larger budgets.
2. Use the `trotter_controlled` builder to create controlled-evolution blocks for QPE; decompose to backend basis.
3. For SBQD on hardware, build Hadamard-test or SWAP-test circuits to estimate overlaps; manage job batching.
4. Use `qiskit_ibm_runtime` EstimatorV2/SamplerV2 for efficient execution and session reuse.

This notebook is a starting point — further automation (job batching, readout calibration, ZNE) can be added.